## 네이버 뉴스 크롤링

In [1]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from pprint import pprint
from tqdm.notebook import tqdm

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 전문 가져오기 >_select 사용
- 네이버 뉴스만 가져와서 결과값 조금 작음 
- 결과 메모장 저장 -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
RESULT_PATH = 'data/'
now = datetime.now() #파일이름 현 시간으로 저장하기

def get_news(n_url):
    news_detail = []

    breq = requests.get(n_url, headers={'User-Agent':'Mozilla/5.0'})
    bsoup = BeautifulSoup(breq.content)

    title = bsoup.find('h2', class_='media_end_head_headline').text
    news_detail.append(title)

    pdate = bsoup.find('span', class_="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME").text[:11]
    news_detail.append(pdate)

    _text = bsoup.find('div', class_='newsct_article _article_body').get_text().replace('\n', " ")
    _text = _text.replace("    동영상 뉴스   ", "")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())
  
    news_detail.append(n_url)
    
    pcompany = bsoup.find('div', class_='copyright').find('p', class_='c_text').text
    pcompany = pcompany.split('All rights reserved.')[0].split('Copyright ⓒ')[1].strip()
    news_detail.append(pcompany)

    return news_detail

def crawler(maxpage,query,s_date,e_date):

    crawled_data = {}
    years = []
    company = []
    title = []
    contents = []
    link = []
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    f = open(f"data/contents_text_{query}.txt", 'w', encoding='utf-8-sig')
    
    for page in tqdm(range(1, maxpage_t+1, 10)):
    
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        req = requests.get(url,
                          headers={'User-Agent':'Mozilla/5.0'})
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont)
        
        for urls in soup.find_all("a", class_="info"):
            try :
                if urls.get_text() == "네이버뉴스":
                    print(urls["href"])
                    news_detail = get_news(urls["href"])
                        # pdate, pcompany, title, btext
                    f.write("{}\t{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[4], news_detail[0], news_detail[2],news_detail[3]))  # new style
                    years.append(news_detail[1])
                    company.append(news_detail[4])
                    title.append(news_detail[0])
                    contents.append(news_detail[2])
                    link.append(news_detail[3])
                    
            except Exception as e:
                print(e)
                continue
        
    crawled_data["years"] = years
    crawled_data["company"] = company
    crawled_data["title"] = title
    crawled_data["contents"] = contents
    crawled_data["link"] = link
    f.close()
    return crawled_data
    
def excel_make(crawled_data):
    data = pd.DataFrame(crawled_data)
    
    xlsx_outputFileName = f'{query}_({s_date}~{e_date}).xlsx'
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8-sig', index=False)


maxpage = input("최대 출력할 페이지수 입력하시오: ")
if not maxpage.isdigit():
    maxpage = input("최대 출력할 페이지수 입력하시오: ")
query = input("검색어 입력: ")
s_date = input("시작날짜 입력(ex) 2021.01.01):")
e_date = input("끝날짜 입력(ex) 2021.05.24):")
crawled_data = crawler(maxpage,query,s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다. 
excel_make(crawled_data) #엑셀로 만들기

최대 출력할 페이지수 입력하시오: 1000
검색어 입력: 갤럭시
시작날짜 입력(ex) 2021.01.01):2022.01.01
끝날짜 입력(ex) 2021.05.24):2022.12.06


  0%|          | 0/1000 [00:00<?, ?it/s]

https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=1
https://n.news.naver.com/mnews/article/023/0003732662?sid=101
https://n.news.naver.com/mnews/article/020/0003466150?sid=101
https://n.news.naver.com/mnews/article/081/0003322592?sid=101
https://n.news.naver.com/mnews/article/088/0000787232?sid=101
https://n.news.naver.com/mnews/article/119/0002664180?sid=101
https://n.news.naver.com/mnews/article/374/0000313153?sid=101
https://n.news.naver.com/mnews/article/421/0006502993?sid=102
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=11
https://n.news.naver.com/mnews/article/008/0004825528?sid=105
https://n.news.naver.com/mnews/article/014/0004935255?sid=105
https://n.news.naver.com/mnews/article/366/0000858351?sid=105
https://n.news.naver.com/mnews/article/030/0003063371?sid=105
https://n.news.nave

https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=221
https://n.news.naver.com/mnews/article/001/0013611921?sid=101
https://n.news.naver.com/mnews/article/029/0002769574?sid=106
'NoneType' object has no attribute 'text'
https://n.news.naver.com/mnews/article/215/0001066450?sid=105
https://n.news.naver.com/mnews/article/030/0003061537?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=231
https://n.news.naver.com/mnews/article/030/0003061537?sid=105
https://n.news.naver.com/mnews/article/030/0003058409?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=241
https://n.news.naver.com/mnews/article/030/0003058409?sid=105
https://n.news.naver.com/mnews/article/079/0003707641?sid=105
https://n

https://n.news.naver.com/mnews/article/648/0000011575?sid=101
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=391
https://n.news.naver.com/mnews/article/366/0000853100?sid=101
https://n.news.naver.com/mnews/article/366/0000855209?sid=105
https://n.news.naver.com/mnews/article/016/0002064948?sid=105
https://n.news.naver.com/mnews/article/032/0003187867?sid=101
https://n.news.naver.com/mnews/article/417/0000870128?sid=105
https://n.news.naver.com/mnews/article/031/0000710762?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=401
https://n.news.naver.com/mnews/article/092/0002272858?sid=105
https://n.news.naver.com/mnews/article/119/0002657108?sid=101
https://n.news.naver.com/mnews/article/366/0000853677?sid=105
https://n.news.naver.com/mnews/article/018/0005363319?sid=105
https://n.news.n

...

https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=8991
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9001
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9011
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/00040

https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9241
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9251
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9261
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/00040

https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9491
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9501
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://n.news.naver.com/mnews/article/076/0003832034?sid=105
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9511
https://n.news.naver.com/mnews/article/018/0005145595?sid=101
https://n.news.naver.com/mnews/article/421/00059

https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9741
https://n.news.naver.com/mnews/article/018/0005145595?sid=101
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9751
https://n.news.naver.com/mnews/article/018/0005145595?sid=101
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9761
https://n.news.naver.com/mnews/article/018/00051

https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9981
https://n.news.naver.com/mnews/article/018/0005145595?sid=101
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101
https://search.naver.com/search.naver?where=news&query=갤럭시&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9991
https://n.news.naver.com/mnews/article/018/0005145595?sid=101
https://n.news.naver.com/mnews/article/421/0005903567?sid=105
https://n.news.naver.com/mnews/article/011/0004018491?sid=101


In [3]:
import pandas as pd

data = pd.read_excel(f'data/{query}_({s_date}~{e_date}).xlsx')
data

,years,company,title,contents,link
0,2022.12.05.,조선일보.,삼성전자 첫 여성 사장 나왔다...갤럭시 마케팅 이끈 이영희 승진 [명단],"언론인 출신 백수현, 박승희 부사장도 나란히 승진 ▲ 삼성전자 이영희 사장\t...",https://n.news.naver.com/mnews/article/023/000...
1,2022.12.05.,동아일보.,삼성전자 첫 여성 사장 탄생…갤럭시 마케팅 이끈 이영희,이영희 DX부문 글로벌마케팅실장 사장 삼성전자가 5일 네트워크와 반도체 사업의 성...,https://n.news.naver.com/mnews/article/020/000...
2,2022.12.06.,서울신문.,‘세계 속의 갤럭시’ 브랜드 각인… 오너家 제외한 첫 여성 사장,삼성 새 역사 쓴 이영희 사장마케팅 전문가로 2007년 영입유니레버·SC존슨·로레알...,https://n.news.naver.com/mnews/article/081/000...
3,2022.12.05.,매일신문.,삼성전자 첫 여성사장 이영희…갤럭시 성공적 마케팅 주역,이영희 삼성전자 DX부문 글로벌마케팅센터장. 연합뉴스삼성에서 오너가 출신이 아닌 첫...,https://n.news.naver.com/mnews/article/088/000...
4,2022.12.05.,데일리안.,삼성전자 첫 여성 CEO 탄생…갤럭시 성공 스토리 쓴 이영희 사장,비오너가 출신 이영희 삼성전자 사장…2012년 부사장 승진 이후 10년 만2007년...,https://n.news.naver.com/mnews/article/119/000...
...,...,...,...,...,...
2717,2022.02.14.,뉴스1.,'갤럭시S22' 지원금 최대 24만원…전작 절반으로 뚝,이통3사 '갤S22' 공시지원금 5.3만~24만원…전작 대비 절반 수준실구매자는 2...,https://n.news.naver.com/mnews/article/421/000...
2718,2022.02.14.,서울경제.,"민팃, 추가 보상 받고 갤럭시 S22 구매",갤럭시 S20·노트10·노트9 15만 원 추가 보상 전국 대형마트 및 통신사 ...,https://n.news.naver.com/mnews/article/011/000...
2719,2022.02.14.,이데일리.,"갤럭시 S22 스마트폰, 마켓컬리 새벽배송으로 구매",[이데일리 전재욱 기자] 온라인 장보기 업체 마켓컬리는 삼성전자 신작 스마트폰 ‘갤...,https://n.news.naver.com/mnews/article/018/000...
2720,2022.02.14.,뉴스1.,'갤럭시S22' 지원금 최대 24만원…전작 절반으로 뚝,이통3사 '갤S22' 공시지원금 5.3만~24만원…전작 대비 절반 수준실구매자는 2...,https://n.news.naver.com/mnews/article/421/000...
